# Accurate Integer Mathematics in Transformers - Train the Model

This CoLab defines and trains a Transformer model that performs integer addition, subtraction and multiplication e.g. 133357+182243=+0315600, 123450-345670=-0123230 and 000345*000823=+283935. Each digit is a separate token. For 6 digit questions, the model is given 14 "question" (input) tokens, and must then predict the corresponding 8 "answer" (output) tokens.

This CoLab trains the model, storing the results to Google Drive.

## Tips for using the Colab
 * You can run and alter the code in this CoLab notebook yourself in Google CoLab ( https://colab.research.google.com/ ).
 * To run the notebook, in Google CoLab, **you will need to** go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.
 * Some graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate.
 * Collapse irrelevant sections with the dropdown arrows.
 * Search the page using the search in the sidebar, not CTRL+F.

# Part 1: Configuration

In [1]:
# Tokens used in vocab. (Token indexes 0 to 9 represent digits 0 to 9)
PLUS_INDEX = 10
MINUS_INDEX = 11
EQUALS_INDEX = 12
MULT_INDEX = 13
DIV_INDEX = 14
MAX_INDEX = DIV_INDEX

In [2]:
# Main configuration class for main model creation and training
class Config():
  #@markdown Main Model
  n_layers: int = 3 #@param
  n_heads: int = 4 #@param

  d_vocab: int = MAX_INDEX+1
  d_model: int = 510
  d_mlp: int = 4 * d_model
  d_head: int = 170
  seed: int = 129000 #@param

  #@markdown Data
  n_digits: int = 6 #@param
  n_ctx: int = 3 * n_digits + 4
  act_fn: str = 'relu'
  batch_size: int = 64 #@param

  #@markdown Optimizer
  n_training_steps: int = 45000 #@param
  lr: float = 0.00008 #@param
  weight_decay: int = 0.1 #@param

  # Save graphs to CoLab temp files as PDF and HTML. Can manually export files for re-use in papers.
  save_graph_to_file: bool = True

  #@markdown Actions

  # Percent of questions that are multiplication, subtraction and addition.
  perc_mult: int = 0 #@param
  perc_sub: int = 90 #@param
  def perc_add(self):
    return max(0, 100 - self.perc_mult - self.perc_sub)

cfg = Config()

In [3]:
# Optional configuration class for inserting a sub-model's weights into the main model before training
class IConfig():
  #@markdown Insert Model
  insert: bool = True #@param
  insert_late: bool = False #@param

  n_layers: int = 2 #@param
  n_heads: int = 3 #@param
  seed: int = 129000 #@param

  #@markdown Data
  n_digits: int = 6 #@param
  n_ctx: int = 3 * n_digits + 4

  #@markdown Optimizer
  n_training_steps: int = 30000 #@param


icfg = IConfig()

In [4]:
def file_name_suffix(digits, layers, heads, d_model, d_head, ctx, seed, training_steps):
  epoch_str = str(training_steps//1000) + "K"
  return '_d{}_l{}_h{}_dm{}_dh{}_ctx{}_seed{}_train{}'.format(digits, layers, heads, d_model, d_head, ctx, seed, epoch_str)

fname_prefix = ''
if icfg.insert:
  fname_prefix = 'ins_'

if cfg.perc_mult == 100:
  fname_prefix += 'mul'
else:
  if cfg.perc_sub == 100:
    fname_prefix += 'sub'
  else:
    if cfg.perc_mult == 0 and cfg.perc_sub == 0:
      fname_prefix += 'add'
    else:
      fname_prefix += 'mix'

main_fname_suffix = fname_prefix + file_name_suffix(cfg.n_digits, cfg.n_layers, cfg.n_heads, cfg.d_model, cfg.d_head, cfg.n_ctx, cfg.seed, cfg.n_training_steps)


def print_config():
  print("%Mult=", cfg.perc_mult, "%Sub=", cfg.perc_sub, "%Add=", cfg.perc_add(), main_fname_suffix)


print_config()

%Mult= 0 %Sub= 90 %Add= 10 ins_mix_d6_l3_h4_dm510_dh170_ctx22_seed129000_train45K


# Part 2: Import libraries
Imports standard libraries. Will ask for access to your Google to write model weightings

In [5]:
from google.colab import drive
from pathlib import Path

In [6]:
# Training saves the trained model weights to a file in your Google Drive. You will need to give permission for this CoLab to access your Google Drive.
# Loading loads the model from your Google Drive. Avoids the say 10mins spent on training the model.

GLOBAL=True
if GLOBAL:
    drive.mount('/content/drive', force_remount=False)
    rootdir=Path('/content/drive/MyDrive/AI/CoLabOutput/')
else:
    rootdir=Path('./')

main_fname_full = main_fname_suffix + '.pth'

main_persist_location = rootdir/f'{main_fname_full}'

print('main model will save to {}'.format(str(main_persist_location)))

Mounted at /content/drive
main model will save to /content/drive/MyDrive/AI/CoLabOutput/ins_mix_d6_l3_h4_dm510_dh170_ctx22_seed129000_train45K.pth


In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")

    %pip install kaleido
    %pip install transformer_lens
    %pip install circuitsvis
    %pip install torchtyping
    %pip install transformers

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━ 469.5/670

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import kaleido
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import tqdm.auto as tqdm
import random
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import matplotlib.pyplot as plt
import circuitsvis as cv

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

# Part 3: Create main_model
This section defines the token embedding / unembedding and creates the model.

In [ ]:
# Embedding / Unembedding

def token_to_char(i):
  if i < 10:
   return str(i)
  if i == PLUS_INDEX:
    return "+"
  if i == MINUS_INDEX:
    return "-"
  if i == EQUALS_INDEX:
    return "="
  if i == MULT_INDEX:
    return "*"
  if i == DIV_INDEX:
    return "\\"
  return "?"

def tokens_to_string(tokens):
    tokens = utils.to_numpy(tokens)
    return "".join([token_to_char(i) for i in tokens[:cfg.n_ctx]])

def string_to_tokens(string, batch: bool=False):
    lookup = {str(i):i for i in range(10)}
    lookup['+']=PLUS_INDEX
    lookup['-']=MINUS_INDEX
    lookup['=']=EQUALS_INDEX
    lookup['*']=MULT_INDEX
    lookup['\\']=DIV_INDEX

    tokens = [lookup[i] for i in string if i not in '\n ']
    if batch:
        return torch.tensor(tokens)[None, :]
    else:
        return torch.tensor(tokens)

In [ ]:
# Transformer creation

# Structure is documented at https://neelnanda-io.github.io/TransformerLens/transformer_lens.html#transformer_lens.HookedTransformerConfig.HookedTransformerConfig
ht_cfg = HookedTransformerConfig(
    n_layers = cfg.n_layers,
    n_heads = cfg.n_heads,
    d_model = cfg.d_model,
    d_head = cfg.d_head,
    d_mlp = cfg.d_mlp,
    act_fn = cfg.act_fn,
    normalization_type = 'LN',
    d_vocab = cfg.d_vocab,
    d_vocab_out = cfg.d_vocab,
    n_ctx = cfg.n_ctx,
    init_weights = True,
    device = "cuda",
    seed = cfg.seed,
)

main_model = HookedTransformer(ht_cfg)

optimizer = optim.AdamW(main_model.parameters(),
                        lr = cfg.lr,
                        weight_decay = cfg.weight_decay,
                        betas = (0.9, 0.98))

max_iter = cfg.n_training_steps
warmup_iter = max_iter // 5
scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=int(warmup_iter))
scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(np.ceil((max_iter-warmup_iter))))
scheduler  = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[int(warmup_iter)])

# Part 4: Loss Function & Data Generator
This section defines the loss function and the training/testing data generator.


In [ ]:
# Loss functions

# Calculate the per-token probability by comparing a batch of prediction "logits" to answer "tokens"
def logits_to_tokens_loss(logits, tokens):
  # Addition answer can have one extra digit than question. Answer also has a +/- sign
  n_answer_digits = cfg.n_digits+2

  # The addition answer digit token probabilities
  ans_logits = logits[:, -(n_answer_digits+1):-1]

  # Convert raw score (logits) vector into a probability distribution.
  # Emphasize the largest scores and suppress the smaller ones, to make them more distinguishable.
  ans_probs = F.log_softmax(ans_logits.to(torch.float64), dim=-1)

  max_prob_tokens = torch.argmax(ans_probs, dim=-1)

  # The addition answer digit tokens
  ans_tokens = tokens[:, -(n_answer_digits):]

  # Extract values from the ans_probs tensor, based on indices from the ans_tokens tensor
  ans_loss = torch.gather(ans_probs, -1, ans_tokens[:, :, None])[..., 0]

  return ans_loss, max_prob_tokens

# Calculate loss as negative of average per-token mean probability
def loss_fn(ans_loss):
  return -ans_loss.mean(0)

In [ ]:
# Define "iterator" data generator function. Invoked using next().
# "Addition" batch entries are formated XXXXX+YYYYY=+ZZZZZZ e.g. 550030+800020=+1350050
# "Subtraction" batch entries are formated XXXXX-YYYYY=-ZZZZZZ e.g. 550030-800020=-0249990, 800020-550030=+0249990
# "Multiplication" batch entries are formated 000XXX*000YYY=+ZZZZZZ e.g. 000345*000678=+233910
def data_generator( ):
    torch.manual_seed(cfg.seed)
    while True:
        #generate a batch of questions (answers calculated below)
        batch = torch.zeros((cfg.batch_size, cfg.n_ctx)).to(torch.int64)
        x = torch.randint(0, 10, (cfg.batch_size, cfg.n_digits))
        y = torch.randint(0, 10, (cfg.batch_size, cfg.n_digits))

        batch_rand = random.randint(1, 100)
        batch_op = MULT_INDEX if batch_rand <= cfg.perc_mult else MINUS_INDEX if batch_rand <= cfg.perc_mult + cfg.perc_sub else PLUS_INDEX

        if batch_op == MULT_INDEX:
          # Convert from NNNNNN*NNNNNN= to 000NNN*000NNN= so answer (product) is NNNNNN
          num_zeros = cfg.n_digits // 2
          for z in range(num_zeros):
            x[:, z] = 0
            y[:, z] = 0

        batch[:, :cfg.n_digits] = x
        batch[:, cfg.n_digits] = batch_op
        batch[:, 1+cfg.n_digits:1+cfg.n_digits*2] = y
        batch[:, 1+cfg.n_digits*2] = EQUALS_INDEX

        # Convert each row into a 5-digit number
        if cfg.n_digits == 5:
          x_values = x[:, 0] * 10000 + x[:, 1] * 1000 + x[:, 2] * 100 + x[:, 3] * 10 + x[:, 4]
          y_values = y[:, 0] * 10000 + y[:, 1] * 1000 + y[:, 2] * 100 + y[:, 3] * 10 + y[:, 4]
        else:
          if cfg.n_digits == 6:
            x_values = x[:, 0] * 100000 + x[:, 1] * 10000 + x[:, 2] * 1000 + x[:, 3] * 100 + x[:, 4] * 10 + x[:, 5]
            y_values = y[:, 0] * 100000 + y[:, 1] * 10000 + y[:, 2] * 1000 + y[:, 3] * 100 + y[:, 4] * 10 + y[:, 5]

        # Elementwise operations giving 1D tensor
        if batch_op == MULT_INDEX:
          answers = x_values * y_values
        else:
          if batch_op == MINUS_INDEX:
            answers = x_values - y_values
          else:
            answers = x_values + y_values

        for i in range(cfg.batch_size):
          answer = answers[i]

          sign = PLUS_INDEX
          if answer < 0:
            sign = MINUS_INDEX
            answer = - answer

          batch[i, 2+cfg.n_digits*2] = sign
          for j in range(cfg.n_digits+1):
            batch[i, cfg.n_ctx-j-1] = answer % 10
            answer = answer // 10
            if answer == 0:
                break

        yield batch.cuda()

In [ ]:
# Initialise the data generator
ds = data_generator()

In [ ]:
# Test data generator
tokens = next(ds)
print(tokens[:3,:])

# Part 5: Read insert_model weights from Google drive (optional)

In [ ]:
if icfg.insert == True:

  insert_fname_full = "add" + file_name_suffix(icfg.n_digits, icfg.n_layers, icfg.n_heads, cfg.d_model, cfg.d_head, icfg.n_ctx, icfg.seed, icfg.n_training_steps) + ".pth"
  insert_persist_location = rootdir/f'{insert_fname_full}'

  ht_cfg = HookedTransformerConfig(
      n_layers = icfg.n_layers,
      n_heads = icfg.n_heads,
      d_model = cfg.d_model, # Assume constant
      d_head = cfg.d_head, # Assume constant
      d_mlp = cfg.d_mlp, # Assume constant
      act_fn = cfg.act_fn, # Assume constant
      normalization_type = 'LN',
      d_vocab = cfg.d_vocab, # Assume constant
      d_vocab_out = cfg.d_vocab, # Assume constant
      n_ctx = icfg.n_ctx,
      init_weights = True, # Assume constant
      device = "cuda",
      seed = icfg.seed,
  )

  insert_model = HookedTransformer(ht_cfg)

  print('insert_model will load from {}'.format(str(insert_persist_location)))
  insert_model.load_state_dict(torch.load(insert_persist_location))
  insert_model.eval()

# Part 6: Insert insert_model weights into untrained main_model (optional)



In [ ]:
# Insert the small model weights into the large model
def do_model_insert(small_model, large_model, start_layer, end_layer, transfer_ln=True, transfer_embeds=True):
    """Args:
    small_model: The model to transfer weights from
    large_model: The model to transfer weights to
    start_layer: The first layer to transfer weights to
    end_layer: The last layer to transfer weights to (Note that this is end-inclusive!)
    """
    small_cfg = {k: v for k,v in small_model.cfg.__dict__.items() if k in ["d_head", "d_mlp", "d_model", "n_heads", "n_layers"]}
    large_cfg = {k: v for k,v in large_model.cfg.__dict__.items() if k in ["d_head", "d_mlp", "d_model", "n_heads", "n_layers"]}

    # Sanity checks for large model > small model
    assert small_cfg["d_model"] == large_cfg["d_model"]
    assert small_cfg["d_head"] == large_cfg["d_head"]
    assert small_cfg["n_layers"] <= large_cfg["n_layers"]
    assert small_cfg["n_heads"] <= large_cfg["n_heads"]
    assert small_cfg["d_mlp"] <= large_cfg["d_mlp"]

    assert 0 <= start_layer < end_layer <= large_cfg["n_layers"] # Make sure start_layer and end_layer are valid
    assert end_layer - start_layer + 1 == small_cfg["n_layers"] # Make sure the number of layers to transfer is correct

    # Transfer heads and MLPs
    for small_layer_no, large_layer_no in enumerate(range(start_layer, end_layer+1)):
        # Transfer Heads
        large_model.blocks[large_layer_no].attn.W_Q.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.W_Q.clone().data
        large_model.blocks[large_layer_no].attn.W_K.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.W_K.clone().data
        large_model.blocks[large_layer_no].attn.W_V.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.W_V.clone().data

        large_model.blocks[large_layer_no].attn.b_Q.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.b_Q.clone().data
        large_model.blocks[large_layer_no].attn.b_K.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.b_K.clone().data
        large_model.blocks[large_layer_no].attn.b_V.data[:small_cfg["n_heads"]] = small_model.blocks[small_layer_no].attn.b_V.clone().data

        # Transfer MLPs
        large_model.blocks[large_layer_no].mlp.W_in.data[:, :small_cfg["d_mlp"]] = small_model.blocks[small_layer_no].mlp.W_in.clone().data
        large_model.blocks[large_layer_no].mlp.b_in.data[:small_cfg["d_mlp"]] = small_model.blocks[small_layer_no].mlp.b_in.clone().data
        large_model.blocks[large_layer_no].mlp.W_out.data[:small_cfg["d_mlp"],] = small_model.blocks[small_layer_no].mlp.W_out.clone().data
        large_model.blocks[large_layer_no].mlp.b_out.data = small_model.blocks[small_layer_no].mlp.b_out.clone().data

    if transfer_ln:
        for small_layer_no, large_layer_no in enumerate(range(start_layer, end_layer+1)):
            large_model.blocks[large_layer_no].ln1.w.data = small_model.blocks[small_layer_no].ln1.w.clone().data
            large_model.blocks[large_layer_no].ln1.b.data = small_model.blocks[small_layer_no].ln1.b.clone().data

        large_model.ln_final.w.data = small_model.ln_final.w.clone().data

    if transfer_embeds:
        large_model.embed.W_E.data = small_model.embed.W_E.clone().data
        large_model.pos_embed.W_pos.data = small_model.pos_embed.W_pos.clone().data
        large_model.unembed.W_U.data = small_model.unembed.W_U.clone().data

In [ ]:
if icfg.insert == True:
  print( "Inserting small_model into main_model.")
  if icfg.insert_late == True:
    # override last few layers
    do_model_insert(small_model=insert_model, large_model=main_model, start_layer=cfg.n_layers - icfg.n_layers, end_layer=cfg.n_layers-1)
  else:
    # override first few layers
    do_model_insert(small_model=insert_model, large_model=main_model, start_layer=0, end_layer=icfg.n_layers-1)

# Part 7: Train add/sub/mult main_model with Infinite Data
Train main_model for n_training_steps, storing train_losses per epoch.

Each training step (of n_training_steps) new training data (a batch of batch_size tokens) is generated and the model is trained and loss calculated on it. No separate "testing" data is needed, as the training data is unique each step. Memorisation of past training data by the model (if any) is minimally beneficial. For 5 digit addition there are 10 billion possible questions, and model training is on ~2 million questions.

In [ ]:
# Train the model
train_losses_list = []
per_token_train_losses_list = []

for epoch in tqdm.tqdm(range(cfg.n_training_steps)):

  tokens = next(ds)
  logits = main_model(tokens)

  per_token_train_losses_raw, _ = logits_to_tokens_loss(logits, tokens)
  per_token_train_losses = loss_fn(per_token_train_losses_raw)
  per_token_train_losses_list.append(utils.to_numpy(per_token_train_losses))

  train_loss = per_token_train_losses.mean()
  train_loss.backward()
  train_losses_list.append(train_loss.item())

  optimizer.step()
  scheduler.step()
  optimizer.zero_grad()

  if epoch % 100 == 0:
    the_loss = train_loss.item()
    print(epoch, the_loss)

In [ ]:
print("Saving main model to Google drive", main_persist_location)
torch.save(main_model.state_dict(), main_persist_location)

# Part 8: Training Loss - Addition and Subtraction

Addition:
- 26Dec23: add_d6_l2_h3_dm510_dh170_ctx22_seed129000_train30K. Loss=9e-9

Subtraction:
- 26Dec23: sub_d6_l2_h3_dm510_dh170_ctx22_seed129000_train30K. Loss=4e-7.
- 26Dec23: sub_d6_l2_h3_dm510_dh170_ctx22_seed129000_train45K. Loss=5.5e-4. Worse! Digits 0 and 3 have most residual loss
- 27Dec23: sub_d6_l2_h4_dm510_dh170_ctx22_seed129000_train30K. Loss=1e-05. Digits 0 & 2 have worst loss.
- 27Dec23: sub_d6_l3_h4_dm510_dh170_ctx22_seed129000_train30K. Loss=7e-5
- 27Dec23: sub_d6_l3_h4_dm510_dh170_ctx22_seed129000_train45K. Loss=1e-6. But per-digit graphs show unhandled exceptions (mostly in digit 2). Definitely not 100%

Inserting trained addition model into untrained subtraction model:
- 28Dec23: inserting add_d6_l2_h3_dm510_dh170_ctx22_seed129000_train30K into ins_sub_d6_l3_h4_dm510_dh170_ctx22_seed129000_train45K with insert_late=False. Loss=1.3e-8! Great

Inserting trained addition model into untrained mix model (10% Add, 90% Sub):
- 28Dec23: inserting add_d6_l2_h3_dm510_dh170_ctx22_seed129000_train30K into ins_mix_d6_l3_h4_dm510_dh170_ctx22_seed129000_train45K with insert_late=False. Loss=1.6e-6 at 36K (of 45K) training.


In [ ]:
print_config()

final_training_loss = round((train_losses_list[-5]+train_losses_list[-4]+train_losses_list[-3]+train_losses_list[-2]+train_losses_list[-1])/5,9)
print( "Final training loss", final_training_loss)

# Part 9: Line Graphs

This section analyses the training loss by graphing it at a high level.

The loss curve for all digits show visible inflection points (bumps), but is too high level to help understand the algorithm.

When this graph is decomposed into 'per digit' graphs, the interesting distinct 'per digit' curves appear, showing each digit is being refined semi-independently, with the model algorithm refining each digit separately.

In [ ]:
def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

# Helper function to plot multiple lines
def lines(raw_lines_list, x=None, mode='lines', labels=None, xaxis='Epoch', yaxis='Loss', title = '', log_y=False, hover=None, **kwargs):

    lines_list = raw_lines_list
    log_suffix = '' if log_y==False else ' (Log)'
    full_title = title + log_suffix

    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    if cfg.save_graph_to_file :
      fig = go.Figure(layout={})
      print(full_title)
    else:
      fig = go.Figure(layout={'title':full_title})

    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis + log_suffix)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    if cfg.save_graph_to_file:
        fig.update_layout(margin=dict(l=10, r=10, t=10, b=10),width=1200,height=300)

    fig.show(bbox_inches="tight")

    if cfg.save_graph_to_file:
        filename = full_title.replace(" ", "").replace("(", "").replace(")", "").replace("&", "").replace(",", "").replace("%", "")   +'.pdf'
        pio.write_image(fig, filename)


title_suffix = 'Digit Loss Curves ' + main_fname_suffix
per_token_losses = np.stack(per_token_train_losses_list, axis=0)

line(train_losses_list,
    title=title_suffix)

lines([per_token_losses[:, i] for i in range(1+cfg.n_digits)]+[train_losses_list],
      labels = [f'digit {i}' for i in range(1+cfg.n_digits)]+['all_digits'],
      title='Per digit'+title_suffix, log_y=False)

lines([per_token_losses[:, i] for i in range(1+cfg.n_digits)]+[train_losses_list],
      labels = [f'digit {i}' for i in range(1+cfg.n_digits)]+['all_digits'],
      title='Per digit'+title_suffix, log_y=True)

for i in range(1+cfg.n_digits):
  print('Final Loss for digit ' + str(i) + ' is ', per_token_losses[-1, i])

# Part 10: Questions Set Up

Create sets of sample questions (by task) to ask the model to predict

In [ ]:
def make_varied_questions():
  q0 = next(ds)
  q1 = next(ds)
  q2 = next(ds)
  q3 = next(ds)

  questions = torch.vstack((q0.cuda(), q1.cuda(), q2.cuda(), q3.cuda()))

  return questions

In [ ]:
verbose = True

In [ ]:
# Build a test batch of random questions
varied_questions = make_varied_questions();


# Run the sample batch, gather the cache
main_model.reset_hooks()
main_model.set_use_attn_result(True)
sample_logits, sample_cache = main_model.run_with_cache(varied_questions.cuda())
print(sample_cache) # Gives names of datasets in the cache
sample_losses_raw, sample_max_prob_tokens = logits_to_tokens_loss(sample_logits, varied_questions.cuda())
sample_loss_mean = utils.to_numpy(loss_fn(sample_losses_raw).mean())
print("Sample Mean Loss", sample_loss_mean)


# attn.hook_z is the "attention head output" hook point name (at a specified layer)
l_attn_hook_z_name = [utils.get_act_name('z', 0, 'a'),utils.get_act_name('z', 1, 'a')] # 'blocks.0.attn.hook_z' etc
sample_attn_z = sample_cache[l_attn_hook_z_name[0]]
print("Sample", l_attn_hook_z_name[0], sample_attn_z.shape) # gives [239, 18, 3, 170] = num_questions, cfg.n_ctx, n_heads, d_head
mean_attn_z = torch.mean(sample_attn_z, dim=0, keepdim=True)
print("Mean", l_attn_hook_z_name[0], mean_attn_z.shape) # gives [1, 18, 3, 170] = 1, cfg.n_ctx, n_heads, d_head


# hook_resid_pre is the "pre residual memory update" hook point name (at a specified layer)
l_hook_resid_pre_name = ['blocks.0.hook_resid_pre','blocks.1.hook_resid_pre']


# hook_resid_post is the "post residual memory update" hook point name (at a specified layer)
l_hook_resid_post_name = ['blocks.0.hook_resid_post','blocks.1.hook_resid_post']
sample_resid_post = sample_cache[l_hook_resid_post_name[0]]
print("Sample", l_hook_resid_post_name[0], sample_resid_post.shape) # gives [239, 18, 510] = num_questions, cfg.n_ctx, d_model
mean_resid_post = torch.mean(sample_resid_post, dim=0, keepdim=True)
print("Mean", l_hook_resid_post_name[0], mean_resid_post.shape) # gives [1, 18, 510] = 1, cfg.n_ctx, d_model


# mlp.hook_post is the "MLP layer" hook point name (at a specified layer)
l_mlp_hook_post_name = [utils.get_act_name('post', 0),utils.get_act_name('post', 1)] # 'blocks.0.mlp.hook_post' etc
sample_mlp_hook_post = sample_cache[l_mlp_hook_post_name[0]]
print("Sample", l_mlp_hook_post_name[0], sample_mlp_hook_post.shape) # gives [239, 18, 2040] = num_questions, cfg.n_ctx, d_model*4
mean_mlp_hook_post = torch.mean(sample_mlp_hook_post, dim=0, keepdim=True)
print("Mean", l_mlp_hook_post_name[0], mean_mlp_hook_post.shape) # gives [1, 18, 2040] = 1, cfg.n_ctx, d_model*4

# Part 11: Attention Patterns
Attention patterns show which token(s) the model's attention heads are paying attention to in each token position of the prediction calculation.

For the default CoLab set up, the  model has 3 attention heads, and performs 5 digit addition. The attention pattern is 18 by 18 squares (as 54321+77779=132100 is 18 tokens). Time proceeds vertically downwards, with one additional token being revealed horizontally at each token position, giving the overall triangle shape. This visualisation provided insights. After the question is fully revealed (at token position 11), each head starts attending to pairs of question digits from left to right (i.e. high-value digits before lower-value digits) giving the “double staircase" shape. The three heads attend to a given digit pair in three different token position, giving a time ordering of heads.

In [ ]:
def show_token_attention_patterns(index, layer, token_at_index, use_case):

  the_tokens = [str(token) for token in token_at_index.tolist()]
  if layer == 0:
    tokens_str = tokens_to_string(token_at_index)
    print("Attention patterns for", tokens_str)

  attention_pattern=sample_cache["pattern", layer, "attn"][index]
  display(cv.attention.attention_patterns(
      tokens=the_tokens,
      attention=attention_pattern,
      #attention_head_names=[f"L{layer}H{i}" for i in range(cfg.n_heads)],
  ))


sample_size = 3

# Show attention patterns for some randomly chosen tokens
for i in range(sample_size):
  for layer in range(cfg.n_layers):
    show_token_attention_patterns(i, layer, tokens[i], "Misc")


In [ ]:
if cfg.save_graph_to_file:

  tokens_str = []
  for i in range(cfg.n_heads):
    one_token_str = []
    for j in tokens[i]:
      one_token_str.append(str(utils.to_numpy(j)))
    tokens_str.append(one_token_str)

  # Refer https://github.com/callummcdougall/CircuitsVis/blob/main/python/circuitsvis/circuitsvis_demo.ipynb

  # html_object = cv.attention.from_cache(
  #    cache = sample_cache,
  #    tokens = tokens_str, # list of list of strings
  #    return_mode = "html",
  #)

  # Create a CoLab file containing the attention pattern(s) in HTML
  #filename = "AttentionPattern" + str(cfg.n_digits) + "Digits" + str(cfg.n_heads) + "Heads.html"
  #with open(filename, "w") as f:
  #    f.write(html_object.data)

  # Manually download the CoLab "html" file and open in your local browser.
  # Install and use the Edge extension "FireShot" to save a portion of the HTML page as a PDF

# Part 12: Run a prediction question

Create way to get model to predict sample question answers and analysis/show results.

In [ ]:
error_count = 0


def predict_experiment_question(questions, the_hook, the_threshold):
  global error_count

  last_loss_mean = 0

  for question_num in range(questions.shape[0]):
    q = questions[question_num]

    main_model.reset_hooks()
    main_model.set_use_attn_result(True)
    the_logits = main_model.run_with_hooks(q.cuda(), return_type="logits", fwd_hooks=the_hook)

    q_2d = q.unsqueeze(0)
    losses_raw, max_prob_tokens = logits_to_tokens_loss(the_logits, q_2d.cuda())
    last_loss_mean = utils.to_numpy(loss_fn(losses_raw).mean())

    # Only show the question if the loss exceeds the threshold (because of the ablated token position)
    if last_loss_mean > the_threshold:
      error_count += 1
      model_answer = tokens_to_string(utils.to_numpy(max_prob_tokens[0]))
      print("Question:", tokens_to_string(q), "ModelAnswer:", model_answer, "Loss:", last_loss_mean)

  return last_loss_mean

In [ ]:
def null_hook(value, hook):
  global verbose

  verbose = False

In [ ]:
# Unit test
qs = next(ds)
the_hook = [(l_attn_hook_z_name[0], null_hook)]
predict_experiment_question(qs, the_hook, 0.01)
print(error_count)

# Part 13 : Is the model 100% accurate?

This is hard to prove. If it does 1M predictions without error, then we assume it is 100%.

This part takes ~90 mins to run for add_5d_2l_3h model.

In [ ]:
def one_million_questions():
  global verbose
  global error_count


  verbose = False

  the_threshold = 0.12
  the_successes = 0
  the_fails = 0

  num_batches = 1000000//cfg.batch_size
  for epoch in range(num_batches):
      tokens = next(ds)

      the_hook = [(l_attn_hook_z_name[0], null_hook)]
      predict_experiment_question(tokens, the_hook, the_threshold)

      if error_count > 0:
        break

      the_successes = the_successes + len(cfg.batch_size)

      if epoch % 100 == 0:
          print("Batch", epoch, "of", num_batches, "#Successes=", the_successes)

  print("successes", the_successes, "num_fails", the_fails)


# Commented out as it takes ~90 minutes to run with cfg.n_layers=2, n_digits=5, train=30K
# one_million_questions()